In [ ]:
# 0. CHECK GPU (Must be run first)
import torch
try:
    print("🔍 Checking GPU Status...")
    !nvidia-smi
    if torch.cuda.is_available():
        print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    else:
        raise RuntimeError("❌ No GPU detected! Please go to Runtime > Change runtime type > T4 GPU")
except Exception as e:
    print(str(e))
    # Stop execution if no GPU
    raise e

# 1. Install Dependencies
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q --upgrade transformers bitsandbytes accelerate
!pip install -q --upgrade "fastapi>=0.115.0,<0.124.0" uvicorn pyngrok nest-asyncio

# 2. Run Server code
import os
import traceback
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from fastapi import FastAPI, Request, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import threading
import asyncio

# --- CONFIG ---
MODEL_NAME = "unsloth/llama-3-8b-Instruct-bnb-4bit" 
NGROK_TOKEN = "ใส่_NGROK_TOKEN_ของคุณตรงนี้"

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

print("⏳ Initializing Model (Phase 1/3)...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

try:
    if not torch.cuda.is_available():
        raise RuntimeError("No GPU available for model loading!")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config, 
        device_map="auto"
    )
    print("✅ Model Loaded Successfully! (Phase 2/3)")
except Exception as e:
    print(f"❌ Model Load Failed: {e}")
    traceback.print_exc()

@app.post("/chat")
async def chat(request: Request):
    try:
        data = await request.json()
        message = data.get("message", "")
        persona = data.get("persona", "") 
        
        print(f"📩 Received: {message[:50]}...")
        
        # --- PROMPT CONSTRUCTION (LLAMA 3 FORMAT) ---
        # Use standard Llama 3 Instruct tags
        input_text = (
            f"<|begin_of_text|>"
            f"<|start_header_id|>system<|end_header_id|>\n\n"
            f"{persona}\n"
            f"<|eot_id|>"
            f"<|start_header_id|>user<|end_header_id|>\n\n"
            f"{message}\n"
            f"<|eot_id|>"
            f"<|start_header_id|>assistant<|end_header_id|>\n\n"
        )
        
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_new_tokens=256,
                tokenizer=tokenizer,
                stop_strings=["<|eot_id|>", "<|end_of_text|>"] # Native stops
            )
        
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Cleanup Llama 3 artifacts
        # Standard decoding might leave the full prompt. We strip it.
        if "assistant" in reply:
            reply = reply.split("assistant")[-1].strip()
        
        # Manual clean cut for any leaked headers
        for stop_word in ["User:", "Mali:", "<|eot_id|>", "<|start_header_id|>"]:
            if stop_word in reply:
                reply = reply.split(stop_word)[0].strip()

        return {"reply": reply}

    except Exception as e:
        print("🔥 generation error (Detail below):")
        traceback.print_exc()
        return {"reply": f"Server Core Error: {str(e)}"}

# Start Server
print("🚀 Starting Server (Phase 3/3)...")
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(8000).public_url
print(f"🔗 Public URL: {public_url}")
print("ให้นำ URL นี้ไปใส่ในตั้งค่า AInote ได้เลยครับ!")

nest_asyncio.apply()

config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
await server.serve()